In [16]:
import cv2 as cv
import numpy as np
import imutils
from matplotlib import pyplot as plt
import easyocr

AttributeError: partially initialized module 'cv2' has no attribute 'gapi_wip_gst_GStreamerPipeline' (most likely due to a circular import)

In [ ]:
! pip uninstall opencv-python

In [7]:
car_cas = cv.CascadeClassifier('C:\\Users\\NRTC Software Team\\Desktop\\Umar Zaib\\vehicle_detection_haarcascades-master\\numberPlateDetection.xml')
img = cv.imread('C:\\Users\\NRTC Software Team\\Desktop\\Umar Zaib\\images.jpg')
cv.imshow('image', img)
cv.waitKey(0)

error: OpenCV(4.5.4) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window.cpp:1274: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvShowImage'


In [8]:
#cap = cv.VideoCapture(video_src)

In [9]:
def detection(frame):
    gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    cars = car_cas.detectMultiScale(frame, 1.1, 1)
    for (x,y,w,h) in cars:
        cv.rectangle(frame,(x,y),(x+w+10,y+h+25),(0,0,255),2)
        #roi_gray = gray[y:y+h, x:x+w] 
        roi_color = frame[y:y+h, x:x+w]
    return frame

In [10]:
gray = cv.cvtColor(pic, cv.COLOR_BGR2GRAY)
#resize = cv.resize(pic, (700,500))
frame = detection(pic)
cv.imshow('pic', frame)
cv.waitKey(0)

error: OpenCV(4.5.4) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window.cpp:1274: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvShowImage'


In [ ]:
plt.subplot(1,1,1), plt.imshow(frame)

In [ ]:
reader = easyocr.Reader(['en'])
result = reader.readtext(frame)
result[0][1]

In [ ]:
gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
cv.imshow('gray',gray)
cv.waitKey(0)

In [ ]:
kernel = np.ones((5,5),np.uint8)
erosion = cv.erode(frame,kernel,iterations = 1)

plt.subplot(1,1,1), plt.imshow(erosion)
plt.title('Morphological Transformation/Erosion'), plt.xticks([]), plt.yticks([])
plt.show()

In [ ]:
image = frame
ratio = image.shape[0] / 500.0
orig = image.copy()
image = imutils.resize(image, height = 500)
 
# convert the image to grayscale, blur it, and find edges
# in the image
gray = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
gray = cv.GaussianBlur(gray, (5, 5), 0)
edged = cv.Canny(gray, 75, 200)
 
# show the original image and the edge detected image
print("STEP 1: Edge Detection")
cv.imshow('org',image)
cv.imshow('edge',edged)
cv.waitKey(0)

In [ ]:
! pip install Pillow
! pip install pytesseract

In [ ]:
img = cv.resize(frame, (620,480) )
gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY) #convert to grey scale
gray = cv.bilateralFilter(gray, 13, 15, 15)
edged = cv.Canny(gray, 30, 200) #Perform Edge detection
cv.imshow('edge',edged)
cv.waitKey(0)

In [ ]:
contours=cv.findContours(edged,cv.RETR_TREE,cv.CHAIN_APPROX_SIMPLE)
contours = imutils.grab_contours(contours)
contours = sorted(contours,key=cv.contourArea, reverse = True)[:10]
screenCnt = None

In [ ]:
for c in contours:
    # approximate the contour
    peri = cv.arcLength(c, True)
    approx = cv.approxPolyDP(c, 0.018  * peri, True)
    # if our approximated contour has four points, then
    # we can assume that we have found our screen
    if len(approx) == 5:
        screenCnt = approx
        break

In [ ]:
if screenCnt is None:
    detected = 0
    print ("No contour detected")
else:
     detected = 1

if detected == 1:
    cv.drawContours(frame, [screenCnt], -1, (0, 0, 255), 3)

In [ ]:
# Masking the part other than the number plate
mask = np.zeros(gray.shape,np.uint8)
new_image = cv.drawContours(mask,[screenCnt],0,255,-1,)
new_image = cv.bitwise_and(frame,frame,mask=mask)
cv.imshow('new',new_image)
cv.waitKey(0)

In [ ]:
# Now crop
(x, y) = np.where(mask == 255)
(topx, topy) = (np.min(x), np.min(y))
(bottomx, bottomy) = (np.max(x), np.max(y))
Cropped = gray[topx:bottomx+1, topy:bottomy+1]

In [ ]:
plt.imshow(Cropped, cmap = 'gray');

In [ ]:
reader = easyocr.Reader(['en'])
result = reader.readtext(Cropped)
result[0][1]

In [ ]:
while True:
    _, frame = cap.read()
    if (type(frame) == type(None)):
        break
    
    gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    frame = detection(gray, frame)
    cv.imshow('video', frame)
    if cv.waitKey(1) & 0xFF == ord('q'):
        break

In [ ]:
cv.destroyAllWindows()